In [1]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s e

In [2]:
import whisper
from moviepy.editor import VideoFileClip

def extract_audio(video_path, audio_path="extracted_audio.wav"):
    print("[INFO] Extracting audio from video...")
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path, logger=None)
    return audio_path

def transcribe_audio(audio_path, translate=False, language="en"):
    print("[INFO] Loading Whisper model...")
    model = whisper.load_model("tiny", device="cpu")
    task_type = "translate" if translate else "transcribe"
    print(f"[INFO] Starting '{task_type}' task...")

    result = model.transcribe(
        audio_path,
        task=task_type,
        language=language,
        fp16=False
    )

    print(f"[INFO] Detected Language: {result['language']}")
    print("\n📄 Transcript:\n")

    return result["text"]

def generate_transcript(video_path, translate=False):
    audio_path = extract_audio(video_path)
    transcript = transcribe_audio(audio_path, translate=translate)
    return transcript


video_path =r"/content/bad.mp4"
generate_transcript(video_path, translate=False)


  if event.key is 'enter':



[INFO] Extracting audio from video...
[INFO] Loading Whisper model...


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 116MiB/s]


[INFO] Starting 'transcribe' task...
[INFO] Detected Language: en

📄 Transcript:



" What's the best curse word sensor? For example, if I say, F***! And it's all right, but it could be better. What about, I can't believe this bullshit! It's a common one for YouTubers, but maybe it's too loud. And if I say, I put the fucking shit in the toaster but the bit didn't bring come out. I mean, you can't go wrong with that one. How about this? I like to eat shit with the friend. No, I said, fork. Hold up, that's not even true last one. Whoa! Fuck, what the shit? She's suck at that baby."

In [3]:
import re

profanity_map = {
    "f***": "fuck", "f***ing": "fucking", "sh**": "shit", "sh***": "shitty",
    "b***h": "bitch", "b***d": "bastard", "b***g": "bastard", "a**": "ass",
    "a**hole": "asshole", "d***": "dick", "d***head": "dickhead", "c***": "cunt",
    "w****": "whore", "sl**": "slut"
}


def restore_profanity(text, mapping):
    for censored, full in sorted(mapping.items(), key=lambda x: -len(x[0])):
        pattern = re.compile(re.escape(censored), re.IGNORECASE)
        text = pattern.sub(full, text)
    return text

def count_profanity(text, offensive_words):
    matches = []
    for word in offensive_words:
        pattern = re.compile(r'\b' + re.escape(word) + r'\b', re.IGNORECASE)
        found = pattern.findall(text)
        matches.extend(found)
    return matches

# List of full offensive words (can expand as needed)
offensive_keywords = [
     "crap","fraud", "scam", "homicide", "manslaughter", "molest", "bribery", "battery", "smuggling", "drug", "gang",
    "cartel", "extort", "vandalism", "arson", "terrorism", "abuse", "violence", "blood", "gore", "stab",
    "shoot", "kill", "bomb", "explode", "hang", "lynch", "beat", "meth", "heroin", "overdose",
    "pervert", "degenerate", "incest", "traffick", "hijack", "kidnap", "prison", "jail", "felony", "hate",
    "bully", "scumbag", "vomit", "snuff", "gangbang", "asshole", "murder", "rape", "assault", "robbery", "bastard", "bullshit",
    "cunt", "dick", "pussy", "cock", "whore", "slut", "damn", "maniac","murder", "psychotic", "methhead", "pedophile", "motherfucker",
    "craphead", "prick", "backstabber", "arse", "threat", "menace", "sniper", "terrorist", "convict", "delinquent",
    "thug", "felon", "narc", "junkie", "crackhead","fuck", "shit", "bitch", "retard", "nigga", "abductor", "rapist", "cheater", "liar", "scoundrel",
    "lowlife", "hatecrime", "corrupt", "smut", "blasphemy", "profanity", "degeneracy", "cruelty", "curse", "harass",
    "torture", "sadist", "necrophilia", "manslaughter", "gangrape", "exploit", "molester", "intimidate", "illegal", "terrorize",
    "invade", "blackmail", "fight", "rage", "anarchy", "outlaw", "misconduct", "slander", "defame", "abuse",
    "mislead", "oppress", "slur", "racist", "sexist", "misogyny", "homophobia", "bigot", "xenophobe", "threaten",
    "punch", "kick", "strangle", "poison", "knife", "rob", "shootout", "blowup", "riot", "loot",
    "mace", "sabotage", "brutality", "explosion", "molotov", "bludgeon", "terror", "insult", "spit", "beatdown",
    "knockout", "ragequit", "psycho", "lunatic", "crook", "conman", "swindle", "embezzle", "rapevictim", "drugdealer",
    "cartelboss", "moneylaunder", "doper", "felonycharge", "stalker", "hacker", "explosive", "gunshot", "threatlevel", "panic",
    "riotgear", "scuffle", "bootlicker", "two-faced", "killjoy", "nitwit", "freak", "jerk", "loser", "creep",
    "weirdo", "nutcase", "moron", "dumbass", "a-hole", "shithead", "nutjob", "twit", "airhead", "buffoon",
    "dunce", "grifter", "tool", "sadistic", "perverse", "obscene", "gross", "barbaric", "verbalabuse", "mentalabuse"
]
# Example input
transcript = generate_transcript(video_path, translate=False)

cleaned_transcript = restore_profanity(transcript, profanity_map)
matched_words = count_profanity(cleaned_transcript, offensive_keywords)
print(cleaned_transcript)

[INFO] Extracting audio from video...
[INFO] Loading Whisper model...
[INFO] Starting 'transcribe' task...
[INFO] Detected Language: en

📄 Transcript:

 What's the best curse word sensor? For example, if I say, fuck! And it's all right, but it could be better. What about, I can't believe this bullshit! It's a common one for YouTubers, but maybe it's too loud. And if I say, I put the fucking shit in the toaster but the bit didn't bring come out. I mean, you can't go wrong with that one. How about this? I like to eat shit with the friend. No, I said, fork. Hold up, that's not even true last one. Whoa! Fuck, what the shit? She's suck at that baby.


In [4]:
if len(matched_words) >=15:
    print(" Result:  Highly Offensive video. Not suitable for children.")
elif len(matched_words)>=10:
    print(" Result: Medium Offensive video. Contain some vurgular content.")
elif len(matched_words)>=5:
    print(" Result: Contain some offensive words.Not Good for small Children")
else:
    print(" Result: Video is okay. No excessive profanity found.")

print(f"\nMatched Words ({len(matched_words)}): {matched_words}")

 Result: Contain some offensive words.Not Good for small Children

Matched Words (7): ['bullshit', 'fuck', 'Fuck', 'shit', 'shit', 'shit', 'curse']


In [5]:
ipc_dict = {
    "murder": {"ipc_section": "302", "description": "Punishment for murder"},
    "rape": {"ipc_section": "376", "description": "Punishment for rape"},
    "assault": {"ipc_section": "351", "description": "Punishment for assault"},
    "theft": {"ipc_section": "378", "description": "Punishment for theft"},
    "robbery": {"ipc_section": "390", "description": "Punishment for robbery"},
    "extortion": {"ipc_section": "384", "description": "Punishment for extortion"},
    "blackmail": {"ipc_section": "503", "description": "Criminal intimidation"},
    "molestation": {"ipc_section": "354", "description": "Assault or criminal force to woman with intent to outrage her modesty"},
    "harassment": {"ipc_section": "509", "description": "Insulting modesty of woman"},
    "acid": {"ipc_section": "326A", "description": "Voluntarily causing grievous hurt by use of acid"},
    "scam": {"ipc_section": "420", "description": "Cheating and dishonestly inducing delivery of property"},
    "bribe": {"ipc_section": "7 (PCA)", "description": "Public servant taking gratification (Prevention of Corruption Act)"},
    "cheat": {"ipc_section": "415", "description": "Cheating"},
    "fake": {"ipc_section": "463", "description": "Forgery"},
    "forgery": {"ipc_section": "465", "description": "Punishment for forgery"},
    "porn": {"ipc_section": "67 IT Act", "description": "Publishing or transmitting obscene material"},
    "terror": {"ipc_section": "16 UAPA", "description": "Terrorist act"},
    "smuggle": {"ipc_section": "135 Customs Act", "description": "Smuggling and duty evasion"},
    "prostitution": {"ipc_section": "8 ITPA", "description": "Procuring person for prostitution"},
    "lynching": {"ipc_section": "302", "description": "Punishment for murder (lynching)"},
    "blasphemy": {"ipc_section": "295A", "description": "Deliberate insult to religion"},
    "riot": {"ipc_section": "146", "description": "Rioting"},
    "sedition": {"ipc_section": "124A", "description": "Sedition (under review)"},
    "hate": {"ipc_section": "153A", "description": "Promoting enmity between groups"},
    "violence": {"ipc_section": "505", "description": "Statements promoting public mischief"},
    "drugs": {"ipc_section": "NDPS Act", "description": "Possession or trafficking of drugs"},
    "abuse": {"ipc_section": "294", "description": "Obscene acts and songs"},
    "slap": {"ipc_section": "352", "description": "Assault or criminal force"},
    "punch": {"ipc_section": "352", "description": "Assault"},
    "kick": {"ipc_section": "352", "description": "Assault"},
    "stab": {"ipc_section": "324", "description": "Voluntarily causing hurt with dangerous weapons"},
    "shoot": {"ipc_section": "307", "description": "Attempt to murder"},
    "kill": {"ipc_section": "302", "description": "Punishment for murder"},
    "explosion": {"ipc_section": "286", "description": "Negligent conduct with explosive substance"},
    "gun": {"ipc_section": "25 Arms Act", "description": "Illegal possession of firearms"},
    "knife": {"ipc_section": "324", "description": "Causing hurt with dangerous weapons"},
    "hostage": {"ipc_section": "364A", "description": "Kidnapping for ransom"},
    "bomb": {"ipc_section": "286", "description": "Negligent conduct with explosives"},
    "trafficking": {"ipc_section": "370", "description": "Trafficking of persons"},
    "cybercrime": {"ipc_section": "66 IT Act", "description": "Cybercrime offenses"},
    "threat": {"ipc_section": "506", "description": "Criminal intimidation"},
    "gang": {"ipc_section": "120B", "description": "Criminal conspiracy"},
    "sexual": {"ipc_section": "354A", "description": "Sexual harassment"},
    "nude": {"ipc_section": "67A IT Act", "description": "Sexually explicit content"},
    "child abuse": {"ipc_section": "POCSO Act", "description": "Protection of children from sexual offences"},
    "domestic violence": {"ipc_section": "498A", "description": "Cruelty by husband or relatives"},
    "manslaughter": {"ipc_section": "304", "description": "Culpable homicide not amounting to murder"},
    "loot": {"ipc_section": "392", "description": "Punishment for robbery"},
    "firearm": {"ipc_section": "25 Arms Act", "description": "Illegal arms possession"},
    "illegal": {"ipc_section": "General", "description": "General illegal activity (context-specific)"},
    "vandalism": {"ipc_section": "427", "description": "Mischief causing damage"},
    "brutal": {"ipc_section": "General", "description": "Descriptive - no direct IPC"},
    "abduct": {"ipc_section": "362", "description": "Abduction"},
    "beating": {"ipc_section": "323", "description": "Voluntarily causing hurt"},
    "exploit": {"ipc_section": "370", "description": "Exploitation in trafficking"},
    "corruption": {"ipc_section": "13 PCA", "description": "Criminal misconduct by public servant"},
    "offend": {"ipc_section": "294", "description": "Obscene acts in public"},
    "fuck": {"ipc_section": "Posco Act", "description": "Profanity / offensive word"},
    "shit": {"ipc_section": "502", "description": "Profanity"},
    "bitch": {"ipc_section": "Offensive", "description": "Profanity / derogatory"},
    "bastard": {"ipc_section": "Offensive", "description": "Profanity"},
    "bullshit": {"ipc_section": "Offensive", "description": "Profanity"}
}


In [6]:
import re

# Sample transcript text
transcript =cleaned_transcript

# Function to match and extract IPC sections
def match_ipc_sections(text, dictionary):
    matches = []
    for word in dictionary:
        pattern = re.compile(r'\b' + re.escape(word) + r'\b', re.IGNORECASE)
        if pattern.search(text):
            match_info = {
                "word": word,
                "ipc_section": dictionary[word]["ipc_section"],
                "description": dictionary[word]["description"]
            }
            matches.append(match_info)
    return matches

# Run matching
results = match_ipc_sections(transcript, ipc_dict)

# Display
print("Matched IPC Sections:")
for item in results:
    print(f"🔹 Word: {item['word']} | IPC Section: {item['ipc_section']} | Description: {item['description']}")


Matched IPC Sections:
🔹 Word: fuck | IPC Section: Posco Act | Description: Profanity / offensive word
🔹 Word: shit | IPC Section: 502 | Description: Profanity
🔹 Word: bullshit | IPC Section: Offensive | Description: Profanity


In [ ]:
import whisper
from moviepy.editor import VideoFileClip

def extract_audio(video_path, audio_path="extracted_audio.wav"):
    print("[INFO] Extracting audio from video...")
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path, logger=None)
    return audio_path

def transcribe_audio(audio_path, translate=False, language="en"):
    print("[INFO] Loading Whisper model...")
    model = whisper.load_model("tiny", device="cpu")
    task_type = "translate" if translate else "transcribe"
    print(f"[INFO] Starting '{task_type}' task...")

    result = model.transcribe(
        audio_path,
        task=task_type,
        language=language,
        fp16=False
    )

    print(f"[INFO] Detected Language: {result['language']}")
    print("\n📄 Transcript:\n")

    return result["text"]

def generate_transcript(video_path, translate=False):
    audio_path = extract_audio(video_path)
    transcript = transcribe_audio(audio_path, translate=translate)
    return transcript


video_path =r"/content/eng.mp4"
generate_transcript(video_path, translate=False)

import re

profanity_map = {
    "f***": "fuck", "f***ing": "fucking", "sh**": "shit", "sh***": "shitty",
    "b***h": "bitch", "b***d": "bastard", "b***g": "bastard", "a**": "ass",
    "a**hole": "asshole", "d***": "dick", "d***head": "dickhead", "c***": "cunt",
    "w****": "whore", "sl**": "slut"
}


def restore_profanity(text, mapping):
    for censored, full in sorted(mapping.items(), key=lambda x: -len(x[0])):
        pattern = re.compile(re.escape(censored), re.IGNORECASE)
        text = pattern.sub(full, text)
    return text

def count_profanity(text, offensive_words):
    matches = []
    for word in offensive_words:
        pattern = re.compile(r'\b' + re.escape(word) + r'\b', re.IGNORECASE)
        found = pattern.findall(text)
        matches.extend(found)
    return matches

# List of full offensive words (can expand as needed)
offensive_keywords = [
     "crap","fraud", "scam", "homicide", "manslaughter", "molest", "bribery", "battery", "smuggling", "drug", "gang",
    "cartel", "extort", "vandalism", "arson", "terrorism", "abuse", "violence", "blood", "gore", "stab",
    "shoot", "kill", "bomb", "explode", "hang", "lynch", "beat", "meth", "heroin", "overdose",
    "pervert", "degenerate", "incest", "traffick", "hijack", "kidnap", "prison", "jail", "felony", "hate",
    "bully", "scumbag", "vomit", "snuff", "gangbang", "asshole", "murder", "rape", "assault", "robbery", "bastard", "bullshit",
    "cunt", "dick", "pussy", "cock", "whore", "slut", "damn", "maniac","murder", "psychotic", "methhead", "pedophile", "motherfucker",
    "craphead", "prick", "backstabber", "arse", "threat", "menace", "sniper", "terrorist", "convict", "delinquent",
    "thug", "felon", "narc", "junkie", "crackhead","fuck", "shit", "bitch", "retard", "nigga", "abductor", "rapist", "cheater", "liar", "scoundrel",
    "lowlife", "hatecrime", "corrupt", "smut", "blasphemy", "profanity", "degeneracy", "cruelty", "curse", "harass",
    "torture", "sadist", "necrophilia", "manslaughter", "gangrape", "exploit", "molester", "intimidate", "illegal", "terrorize",
    "invade", "blackmail", "fight", "rage", "anarchy", "outlaw", "misconduct", "slander", "defame", "abuse",
    "mislead", "oppress", "slur", "racist", "sexist", "misogyny", "homophobia", "bigot", "xenophobe", "threaten",
    "punch", "kick", "strangle", "poison", "knife", "rob", "shootout", "blowup", "riot", "loot",
    "mace", "sabotage", "brutality", "explosion", "molotov", "bludgeon", "terror", "insult", "spit", "beatdown",
    "knockout", "ragequit", "psycho", "lunatic", "crook", "conman", "swindle", "embezzle", "rapevictim", "drugdealer",
    "cartelboss", "moneylaunder", "doper", "felonycharge", "stalker", "hacker", "explosive", "gunshot", "threatlevel", "panic",
    "riotgear", "scuffle", "bootlicker", "two-faced", "killjoy", "nitwit", "freak", "jerk", "loser", "creep",
    "weirdo", "nutcase", "moron", "dumbass", "a-hole", "shithead", "nutjob", "twit", "airhead", "buffoon",
    "dunce", "grifter", "tool", "sadistic", "perverse", "obscene", "gross", "barbaric", "verbalabuse", "mentalabuse"
]
# Example input
transcript = generate_transcript(video_path, translate=False)

cleaned_transcript = restore_profanity(transcript, profanity_map)
matched_words = count_profanity(cleaned_transcript, offensive_keywords)
print(cleaned_transcript)

if len(matched_words) >=15:
    print(" Result:  Highly Offensive video. Not suitable for children.")
elif len(matched_words)>=10:
    print(" Result: Medium Offensive video. Contain some vurgular content.")
elif len(matched_words)>=5:
    print(" Result: Contain some offensive words.Not Good for small Children")
else:
    print(" Result: Video is okay. No excessive profanity found.")

print(f"\nMatched Words ({len(matched_words)}): {matched_words}")
ipc_dict = {
    "murder": {"ipc_section": "302", "description": "Punishment for murder"},
    "rape": {"ipc_section": "376", "description": "Punishment for rape"},
    "assault": {"ipc_section": "351", "description": "Punishment for assault"},
    "theft": {"ipc_section": "378", "description": "Punishment for theft"},
    "robbery": {"ipc_section": "390", "description": "Punishment for robbery"},
    "extortion": {"ipc_section": "384", "description": "Punishment for extortion"},
    "blackmail": {"ipc_section": "503", "description": "Criminal intimidation"},
    "molestation": {"ipc_section": "354", "description": "Assault or criminal force to woman with intent to outrage her modesty"},
    "harassment": {"ipc_section": "509", "description": "Insulting modesty of woman"},
    "acid": {"ipc_section": "326A", "description": "Voluntarily causing grievous hurt by use of acid"},
    "scam": {"ipc_section": "420", "description": "Cheating and dishonestly inducing delivery of property"},
    "bribe": {"ipc_section": "7 (PCA)", "description": "Public servant taking gratification (Prevention of Corruption Act)"},
    "cheat": {"ipc_section": "415", "description": "Cheating"},
    "fake": {"ipc_section": "463", "description": "Forgery"},
    "forgery": {"ipc_section": "465", "description": "Punishment for forgery"},
    "porn": {"ipc_section": "67 IT Act", "description": "Publishing or transmitting obscene material"},
    "terror": {"ipc_section": "16 UAPA", "description": "Terrorist act"},
    "smuggle": {"ipc_section": "135 Customs Act", "description": "Smuggling and duty evasion"},
    "prostitution": {"ipc_section": "8 ITPA", "description": "Procuring person for prostitution"},
    "lynching": {"ipc_section": "302", "description": "Punishment for murder (lynching)"},
    "blasphemy": {"ipc_section": "295A", "description": "Deliberate insult to religion"},
    "riot": {"ipc_section": "146", "description": "Rioting"},
    "sedition": {"ipc_section": "124A", "description": "Sedition (under review)"},
    "hate": {"ipc_section": "153A", "description": "Promoting enmity between groups"},
    "violence": {"ipc_section": "505", "description": "Statements promoting public mischief"},
    "drugs": {"ipc_section": "NDPS Act", "description": "Possession or trafficking of drugs"},
    "abuse": {"ipc_section": "294", "description": "Obscene acts and songs"},
    "slap": {"ipc_section": "352", "description": "Assault or criminal force"},
    "punch": {"ipc_section": "352", "description": "Assault"},
    "kick": {"ipc_section": "352", "description": "Assault"},
    "stab": {"ipc_section": "324", "description": "Voluntarily causing hurt with dangerous weapons"},
    "shoot": {"ipc_section": "307", "description": "Attempt to murder"},
    "kill": {"ipc_section": "302", "description": "Punishment for murder"},
    "explosion": {"ipc_section": "286", "description": "Negligent conduct with explosive substance"},
    "gun": {"ipc_section": "25 Arms Act", "description": "Illegal possession of firearms"},
    "knife": {"ipc_section": "324", "description": "Causing hurt with dangerous weapons"},
    "hostage": {"ipc_section": "364A", "description": "Kidnapping for ransom"},
    "bomb": {"ipc_section": "286", "description": "Negligent conduct with explosives"},
    "trafficking": {"ipc_section": "370", "description": "Trafficking of persons"},
    "cybercrime": {"ipc_section": "66 IT Act", "description": "Cybercrime offenses"},
    "threat": {"ipc_section": "506", "description": "Criminal intimidation"},
    "gang": {"ipc_section": "120B", "description": "Criminal conspiracy"},
    "sexual": {"ipc_section": "354A", "description": "Sexual harassment"},
    "nude": {"ipc_section": "67A IT Act", "description": "Sexually explicit content"},
    "child abuse": {"ipc_section": "POCSO Act", "description": "Protection of children from sexual offences"},
    "domestic violence": {"ipc_section": "498A", "description": "Cruelty by husband or relatives"},
    "manslaughter": {"ipc_section": "304", "description": "Culpable homicide not amounting to murder"},
    "loot": {"ipc_section": "392", "description": "Punishment for robbery"},
    "firearm": {"ipc_section": "25 Arms Act", "description": "Illegal arms possession"},
    "illegal": {"ipc_section": "General", "description": "General illegal activity (context-specific)"},
    "vandalism": {"ipc_section": "427", "description": "Mischief causing damage"},
    "brutal": {"ipc_section": "General", "description": "Descriptive - no direct IPC"},
    "abduct": {"ipc_section": "362", "description": "Abduction"},
    "beating": {"ipc_section": "323", "description": "Voluntarily causing hurt"},
    "exploit": {"ipc_section": "370", "description": "Exploitation in trafficking"},
    "corruption": {"ipc_section": "13 PCA", "description": "Criminal misconduct by public servant"},
    "offend": {"ipc_section": "294", "description": "Obscene acts in public"},
    "fuck": {"ipc_section": "Posco Act", "description": "Profanity / offensive word"},
    "shit": {"ipc_section": "Offensive", "description": "Profanity"},
    "bitch": {"ipc_section": "Offensive", "description": "Profanity / derogatory"},
    "bastard": {"ipc_section": "Offensive", "description": "Profanity"},
    "bullshit": {"ipc_section": "Offensive", "description": "Profanity"}
}

# Sample transcript text
transcript =cleaned_transcript

# Function to match and extract IPC sections
def match_ipc_sections(text, dictionary):
    matches = []
    for word in dictionary:
        pattern = re.compile(r'\b' + re.escape(word) + r'\b', re.IGNORECASE)
        if pattern.search(text):
            match_info = {
                "word": word,
                "ipc_section": dictionary[word]["ipc_section"],
                "description": dictionary[word]["description"]
            }
            matches.append(match_info)
    return matches

# Run matching
results = match_ipc_sections(transcript, ipc_dict)

# Display
print("Matched IPC Sections:")
for item in results:
    print(f"🔹 Word: {item['word']} | IPC Section: {item['ipc_section']} | Description: {item['description']}")


[INFO] Extracting audio from video...
[INFO] Loading Whisper model...
[INFO] Starting 'transcribe' task...
[INFO] Detected Language: en

📄 Transcript:

[INFO] Extracting audio from video...
[INFO] Loading Whisper model...
[INFO] Starting 'transcribe' task...
[INFO] Detected Language: en

📄 Transcript:

 I've got my driver's license last week, just like we always talked about, because you're so excited for me. To finally drive up to your house, but today I drove through this summer. Crying because you weren't around. And you're probably with that blind girl who always made me doubt. She's so much older than me. She's everything I'm insecure about. Yet today I drove through this summer. It's how could I ever love someone else? And I know we weren't perfect, but I've never felt this way for no one. And I just can imagine how you could be so gay, not that I would come. Guess you didn't mean what you're worried that's so about me. She's such forever now, I drive alone past your street. And 